<a href="https://colab.research.google.com/github/Jahnvi99/qWiki-Search/blob/main/qWiki_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11697 sha256=f51334dac18f80d2886df9b9b62a00e1416686c3d9fd898b24dec5d7ab8d902c
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


#Importing libraries

In [2]:
import re
import nltk
import numpy as np
import wikipedia
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
nltk.download('punkt')
import sklearn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
sklearn.__version__

'0.22.2.post1'

#Finding relevant wikipedia pages

In [4]:
query = input('Enter query: ')
n = int(input('Enter number of sentences to be returned: '))

titles = wikipedia.search(query, results=5)
if not titles:
	print ('The given query does not match any wikipedia page.')
	exit()
titles 

Enter query: Narendra Modi
Enter number of sentences to be returned: 5


['Narendra Modi',
 'Narendra Modi Stadium',
 'PM Narendra Modi',
 'Premiership of Narendra Modi',
 'List of international prime ministerial trips made by Narendra Modi']

#Extracting temporal sentences based only on year

In [5]:
data = []
for title in titles:
    try:
        raw_data = wikipedia.page(title).content.replace('\n', ' ')
        processed_data = re.sub('== References ==.+|== See Also ==.+', '', raw_data) #removing irrelevant data from content
        data.append(processed_data)
    except Exception:
        pass

articles = [nltk.sent_tokenize(text) for text in data]
temporal_sentences = []
for article in articles:
    for sentence in article:
        if re.search(r'\d{4}', sentence) is not None:               
            temporal_sentences.append(re.sub('=+.+?=+', '', sentence)) #removing all headings and subheadings

#Creating the document word matrix A

In [6]:
len(temporal_sentences)

297

In [7]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(temporal_sentences)
tfidf.shape

(297, 2052)

#Creating the model to calculate A=WH

In [8]:
nmf_model = NMF(n_components=1, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
W = nmf_model.transform(tfidf)

#Selecting top documents based on the score from topic to document matrix(W)

In [9]:
top_sent_indices = np.argsort( W[:,0] )[::-1][0:n]
best_sentences = [temporal_sentences[sent_index] for sent_index in top_sent_indices]

#Sort and print the final output




In [10]:
  output = []
for line in best_sentences:
    year = re.search(r'\d{4}', line).group()
    output.append((year, line))
sorted_output = sorted(output, key=lambda tup: tup[0], reverse = True)
for item in sorted_output:
	print(item)

('2018', '  On 13 October 2018, Modi was renamed as the BJP candidate for prime minister for the 2019 general election.')
('2014', 'The following is a list of international prime ministerial trips made by Narendra Modi since he became the Prime Minister of India following the 2014 Indian general election.')
('2014', '  After the Bharatiya Janata Party led National Democratic Alliance won a landslide in the 2014 Lok Sabha election, Narendra Modi was sworn in as the Prime Minister of India on 26 May 2014.')
('2014', 'Narendra Modi was sworn in as the Prime Minister of India on 26 May 2014 at the Rashtrapati Bhavan.')
('2013', "     In September 2013 Modi was named the BJP's candidate for prime minister ahead of the 2014 Lok Sabha election.")


In [ ]:
import re
import nltk
import numpy as np
import wikipedia
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

nltk.download('punkt')

def qwiki_search(query, n):
  titles = wikipedia.search(query, results=5)
  if not titles:
    print ('The given query does not match any wikipedia page.')
    exit()
  data = []
  for title in titles:
      try:
          raw_data = wikipedia.page(title).content.replace('\n', ' ')
          processed_data = re.sub('== References ==.+|== See Also ==.+', '', raw_data)#removing irrelevant data from content
          data.append(processed_data)
      except Exception:
          pass

  articles = [nltk.sent_tokenize(text) for text in data]
  temporal_sentences = []
  for article in articles:
      for sentence in article:
          if re.search(r'\d{4}', sentence) is not None:               
              temporal_sentences.append(re.sub('=+.+?=+', '', sentence))
  tfidf_vectorizer = TfidfVectorizer(stop_words='english')
  tfidf = tfidf_vectorizer.fit_transform(temporal_sentences)

 
  nmf_model = NMF(n_components=1, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

  W = nmf_model.transform(tfidf)

 
  top_sent_indices = np.argsort( W[:,0] )[::-1][0:n]
  best_sentences = [temporal_sentences[sent_index] for sent_index in top_sent_indices]

 
  output = []
  for line in best_sentences:
      year = re.search(r'\d{4}', line).group()
      output.append((year, line))
  sorted_output = sorted(output, key=lambda tup: tup[0], reverse = True)
  return sorted_output


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
if __name__ == "__main__":
  query = input('Enter query: ')
  n = int(input('Enter number of sentences to be returned: '))
  sentences = qwiki_search(query, n)
  for item in sentences:
    print(item)


Enter query: Google
Enter number of sentences to be returned: 5
('2017', 'In September 2017, the Google Search app on iOS was updated to feature the same functionality.')
('2013', 'An Android app for My Maps, initially released in March 2013 under the name Google Maps Engine Lite, is also available.')
('2012', 'In December 2012, the Google Maps application was separately made available in the App Store, after Apple removed it from its default installation of the mobile operating system version iOS 6 in September 2012.On January 29, 2013, Google Maps was updated to include a map of North Korea.')
('2012', 'However, with the announcement of iOS 6 in June 2012, Apple announced that they had created their own Apple Maps mapping service, which officially replaced Google Maps when iOS 6 was released on September 19, 2012.')
('2007', '  My Maps is a feature in Google Maps launched in April 2007 that enables users to create custom maps for personal use or sharing.')
